In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../examples/actuation_sparsification/half_vase_opened.msh')
mat = elastic_rods.RodMaterial('+', 20000, 0.3, [2.0, 2.0, 0.2, 0.2], stiffAxis=elastic_rods.StiffAxis.D1)
l.setMaterial(mat)

jdo = l.dofOffsetForJoint(l.centralJoint())
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, l.averageJointAngle, fixedVars=fixedVars)
view = LinkageViewer(l, width=1024)
view.setCameraParams(((0.4075185454716348, 3.1892642201046715, 0.3099480992441177),
 (0.15364528336486324, 0.2839547329660347, 0.9464474821805594),
 (0.0, 0.0, 0.0)))
view.show()

In [ ]:
jointIdxs, stressValues, parametricCoord = l.rodStresses()

In [35]:
print("\n".join(["\t".join(map(str, jidxs)) for jidxs in jointIdxs]), file=open('rod_joints.txt', 'w'))

In [31]:
resolution = 1000
normalize = plt.Normalize(vmin=min(min(stressValues)), vmax=max(max(stressValues)))
cmap = plt.cm.magma

for ri in range(len(jointIdxs)):
    s = stressValues[ri]
    p = parametricCoord[ri]
    # add zero stress samples at the beginning and end of the rod
    s.insert(0, 0)
    s.append(0)
    p.insert(0, 0)
    p.append(np.ceil(p[-1]))
    interpolated_stresses = np.interp(np.linspace(0, p[-1], resolution), p, s)
    image = cmap(normalize(np.array(interpolated_stresses).reshape(1, resolution)))
    plt.imsave('stress_{}.png'.format(ri), image)